<a href="https://colab.research.google.com/github/liz-lewis-manchester/CNM_2025_group_01/blob/testing-solver/Test_case_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML

class AdvectionSolver:
    """
    Finite difference solver for the 1D advection equation:
    d(theta)/dt + U(x)*d(theta)/dx = 0
    Implicit upwind (BTBS) scheme
    """

    def __init__(self, L, dx, dt, T, U):
        self.L = L
        self.dx = dx
        self.dt = dt
        self.T = T

        # Spatial grid
        self.x_new = np.arange(0, L + dx, dx)
        self.num_points = len(self.x_new)
        self.num_time_steps = int(self.T / self.dt)

        # Allow constant or spatially varying velocity
        if np.isscalar(U):
            self.U = U * np.ones(self.num_points)
        else:
            self.U = np.array(U)

        if np.any(self.U <= 0):
            raise ValueError("Velocity must be positive everywhere.")

    # This was previously an incorrectly defined global 'perturbation' function.
    # It has been moved into the class as '_solve_implicit' and its parameters adjusted.
    def _solve_implicit(self, theta_initial, boundary_theta):
        print("Using Implicit Upwind Scheme with Variable Velocity")

        theta = np.copy(theta_initial)
        N = self.num_points

        theta_history = np.zeros((self.num_time_steps + 1, N))
        theta_history[0, :] = theta

        # Courant number (now spatially varying)
        C = self.U * self.dt / self.dx

        for n in range(self.num_time_steps):
            theta_next = np.zeros_like(theta)

            # Boundary condition
            theta_next[0] = boundary_theta[n]

            # Forward substitution
            for i in range(1, N):
                theta_next[i] = (theta[i] + C[i] * theta_next[i-1]) / (1 + C[i])

            theta = theta_next
            theta_history[n + 1, :] = theta

        return theta_history

    def compute_solution(self, theta_initial, boundary_theta):
        return self._solve_implicit(theta_initial, boundary_theta)


# Case 5
L = 20
T = 300
dx = 0.2
dt = 10

n = int(T/dt) +1
# Fixed 'base value' to 'base_value' in parameter name
def perturbation(length, base_value, degree):
  arr = np.full(length, base_value)
  noise = np.random.uniform(-degree, degree, length)
  perturbed_arr = arr * (1+noise)
  return perturbed_arr
x_grid = np.arange(0, L + dx, dx)
U = perturbation(len(x_grid), 0.1, 0.1)
# -------------------------------------------------
# Run solver
# -------------------------------------------------
# Defining x_grid, as it was used but not defined in the cell previously.
# It's assumed to be the same as the spatial grid defined in AdvectionSolver.


solver = AdvectionSolver(L, dx, dt, T, U) # U_profile is used here, not the U defined above. Consider if you meant to use 'U' instead.

theta_initial = np.zeros_like(x_grid)
# Removed: theta_initial = boundary_theta # This line was overwriting the initial condition with a time-series boundary condition.
boundary_theta = np.full(int(T/dt) + 1, 250.0)
result = solver.compute_solution(theta_initial, boundary_theta)


# -------------------------------------------------
# Animation
# -------------------------------------------------
fig, ax = plt.subplots(figsize=(4, 3))
line, = ax.plot([], [], lw=2)

ax.set_xlim(0, L)
ax.set_ylim(0, 300)
ax.set_xlabel("x (m)", fontsize=8)
ax.set_ylabel("Concentration (µg/m³)", fontsize=8)

def init():
    line.set_data([], [])
    return line,

def animate(t):
    line.set_data(x_grid, result[t])
    ax.set_title(f"Concentration at t = {t*dt} s", fontsize=10)
    return line,

anim = matplotlib.animation.FuncAnimation(
    fig, animate, init_func=init, frames=len(result), blit=True
)

plt.close(fig)
display(HTML(anim.to_jshtml()))

Using Implicit Upwind Scheme with Variable Velocity
